In [1]:
import numpy as np
import pandas as pd
import random


def isolation_indeoutlets(assignments, outlets):
    total_group_democrat = sum(1 for a in assignments if a[2] == 'D')
    total_group_republican = sum(1 for a in assignments if a[2] == 'R')

    isolation_sum_d = 0
    isolation_sum_r = 0

    for outlet in outlets:
        if len(outlet) == 1: #Outlet level
            count_group_d = sum(1 for a in assignments if a[0] == outlet[0] and a[2] == 'D')
            count_group_r = sum(1 for a in assignments if a[0] == outlet[0] and a[2] == 'R')
            total_in_outlet = sum(1 for a in assignments if a[0] == outlet[0])

        else: #Article level
            count_group_d = sum(1 for a in assignments if a[0] == outlet[0] and a[2] == 'D')
            count_group_r = sum(1 for a in assignments if a[0] == outlet[0] and a[2] == 'R')
            total_in_outlet = sum(1 for a in assignments if a[:2] == outlet)
        
        if total_in_outlet > 0:
            isolation_sum_d += (count_group_d / total_group_democrat) * (count_group_d / total_in_outlet)
            isolation_sum_r += (count_group_r / total_group_republican) * (count_group_d / total_in_outlet)
    
    return isolation_sum_d - isolation_sum_r


def run_simulation(outlets, articles, democrats, republicans, num_simulations):
    
    segregation_results = {'segregation_index_outlet': [], 'segregation_index_article': []}
    
    for _ in range(num_simulations):
        assignments = [(random.randint(0, outlets-1), random.randint(0, articles-1), 'D') for _ in range(democrats)] + \
                  [(random.randint(0, outlets-1), random.randint(0, articles-1), 'R') for _ in range(republicans)]

        segregation_index_outlet = isolation_indeoutlets(assignments,  [(outlet,) for outlet in range(outlets)])
        segregation_index_article = isolation_indeoutlets(assignments,  [(outlet,article) for outlet in range(outlets) for article in range(articles) ])
            
        segregation_results['segregation_index_outlet'].append(segregation_index_outlet)
        segregation_results['segregation_index_article'].append(segregation_index_article)
    
    
    return pd.DataFrame(segregation_results)

outlet_no = 5
article_no = 3
democrat_no = 5
republican_no = 5
num_simulations = 1000 
segregation_levels = run_simulation(outlet_no, article_no, democrat_no, republican_no, num_simulations)


In [2]:
#Display the whole dataframe
segregation_levels

,segregation_index_outlet,segregation_index_article
0,0.266667,0.2
1,0.700000,3.2
2,0.266667,1.1
3,0.533333,2.8
4,0.300000,3.0
...,...,...
995,0.533333,1.5
996,0.300000,3.0
997,0.200000,0.2
998,0.133333,-0.1


In [3]:
#Print the mean segregation index for both outlet and article
print(segregation_levels.mean())

segregation_index_outlet     0.396561
segregation_index_article    1.215883
dtype: float64


In [5]:
#Export the dataframe to an excel file
segregation_levels.to_excel('segregation_index.xlsx', index=False)